In [1]:
%load_ext autoreload
%autoreload 2

In [143]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial.distance import cosine
import scipy
from ggplot import *
import os


import object_class as oc
from object_vectors import *
import compare_vectors as cv

from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")

In [137]:
all_bl = pd.read_csv("../data/all_basiclevel.csv")
all_bl = all_bl[all_bl["speaker"] != "CHI"]
bl = all_bl['basic_level'].value_counts().to_frame()
bl['word'] = bl.index
bl.columns = ['bl_freq', 'word']

In [150]:
wordvecs = oc.wordvecs(seedlings_categories, "../../../../object_centrality/data_local/dict_glove_42b_300", 
                                             "../../../../object_centrality/data_local/glove_unit_42b_300.npy")
wordvecs.sort(key=lambda x: x[0])

In [52]:
image_vecs = []

for x in seedlings_categories:
    result = oc.find_central_vector(x)
    image_vecs.append((x, result[0]))
    
image_vecs.sort(key=lambda x: x[0])

In [156]:
def generate_pairwise(wordvecs, image_vecs):
    wordvecs.sort(key=lambda x: x[0])
    image_vecs.sort(key=lambda x: x[0])
    
    animate = ["puppy", "duck", "elephant", "pig", "monkey", "giraffe",
                 "bear", "baby", "fish", "frog", "cow", "dog", "cat"]
    results = []
    for idx1, word1 in enumerate(sorted(seedlings_categories)):
        img_vec1 = image_vecs[idx1]
        wrd_vec1 = wordvecs[idx1]
        for idx2, word2 in enumerate(sorted(seedlings_categories)):
            img_vec2 = image_vecs[idx2]
            wrd_vec2 = wordvecs[idx2]
            anim = "mixed"
            if word1 in animate and word2 in animate:
                anim = "animate"
            elif word1 not in animate and word2 not in animate:
                anim = "inanimate"
            if word1 != word2:
                results.append([word1, word2, cosine(img_vec1[1], img_vec2[1]), cosine(wrd_vec1[1], wrd_vec2[1]), anim])

    pairwise = pd.DataFrame(results, columns=["X1", "word", "cos_img", "cos_word", "animate"])
    return pairwise

In [152]:
pairwise = generate_pairwise(wordvecs, image_vecs)
pairwise.to_csv("pairwise_distances.csv", index=False)

In [54]:
df2 = pd.DataFrame()

for idx, group in pairwise.groupby("word"):
    group["zscore_img"] = scipy.stats.zscore(group.cos_img)
    group["zscore_word"] = scipy.stats.zscore(group.cos_word)
    df2 = df2.append(group)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
overlaps = df2.query("(zscore_word <= -1) & (zscore_img <= -1)")
overlaps.to_csv("overlapping_pairwise.csv", index=False)

In [67]:
words = {}

for idx, item in df2.iterrows():
    avg_zscore = round(float((item.zscore_img+item.zscore_word))/2, 2)
    if item.word not in words:
        if item.zscore_img <= -1:
            words[item.word] = [[(item.X1, avg_zscore)],[]]
        if item.zscore_word <= -1:
            words[item.word] = [[],[(item.X1, avg_zscore)]]
    else:
        if item.zscore_img <= -1:
            words[item.word][0].append((item.X1, avg_zscore))
        if item.zscore_word <= -1:
            words[item.word][1].append((item.X1, avg_zscore))
            


In [68]:
overlap_ratios = {}
for key, values in words.items():
    intersect = set(values[0]) & set(values[1])
    n = len(intersect)
    overlap_ratios[key] = [round(float(n)/len(set(values[0]+values[1])), 3), 
                           [list(x) + [True] for x in intersect],
                           [list(x) + [False] for x in values[0] if x not in intersect],
                           [list(x) + [False] for x in values[1] if x not in intersect]]
    

In [76]:
overlap_ratios_list = [(x, y[0],
                        sorted(list(y[1]), key=lambda x: x[1]), 
                        sorted(list(y[2]), key=lambda x: x[1]),
                        sorted(list(y[3]), key=lambda x: x[1]),) 
                       for x, y in overlap_ratios.items()]

In [78]:
overlap_table = pd.DataFrame(overlap_ratios_list, columns = ["word", "overlap_ratio", "overlap_words", "img_neighbors", "word_neighbors"])

In [79]:
overlap_table

,word,overlap_ratio,overlap_words,img_neighbors,word_neighbors
0,apple,0.167,"[[egg, -1.18, True]]","[[puppy, -0.31, False]]","[[milk, -0.66, False], [car, -0.6, False], [bo..."
1,fish,0.250,"[[duck, -1.22, True], [pig, -1.16, True]]","[[cow, -1.15, False], [monkey, -0.45, False], ...","[[frog, -1.09, False], [egg, -0.91, False], [w..."
2,pig,0.125,"[[monkey, -1.28, True]]","[[puppy, -1.06, False], [fish, -0.98, False], ...","[[cow, -1.7, False], [duck, -1.09, False], [fr..."
3,duck,0.143,"[[fish, -1.31, True]]","[[giraffe, -0.2, False]]","[[cow, -1.41, False], [pig, -1.41, False], [eg..."
4,chair,0.000,[],"[[frog, -0.79, False], [spoon, -0.26, False], ...","[[baby, -1.58, False], [ball, -0.96, False], [..."
5,milk,0.200,"[[water, -2.0, True]]",[],"[[bottle, -2.12, False], [egg, -1.43, False], ..."
6,monkey,0.375,"[[cow, -1.49, True], [frog, -1.43, True], [pig...","[[giraffe, -1.12, False], [puppy, -1.0, False]...","[[elephant, -1.13, False], [cat, 0.03, False]]"
7,cup,0.143,"[[egg, -1.32, True]]","[[duck, -0.79, False], [fish, -0.39, False]]","[[spoon, -1.25, False], [milk, -0.93, False], ..."
8,frog,0.100,"[[monkey, -1.53, True]]","[[cow, -1.21, False], [giraffe, -1.03, False],...","[[elephant, -1.1, False], [fish, -0.94, False]..."
9,book,0.000,[],"[[fish, -0.65, False], [monkey, -0.57, False],...","[[car, -1.38, False], [baby, -1.25, False], [d..."


In [80]:
overlap_table.sort_values(by="word").to_csv("overlap_table.csv", index=False)

In [100]:
neighbor_counts = []
for idx, row in overlap_table.iterrows():
    neighbor_counts.append([row.word, len(row.overlap_words), len(row.img_neighbors), len(row.word_neighbors)])
    
neighbors = pd.DataFrame(neighbor_counts, 
                         columns = ["word", "overlap_count", 
                                    "image_count", "word_count"])

In [104]:
max_img_neighbors = neighbors.image_count.max()
max_wrd_neighbors = neighbors.word_count.max()
min_img_neighbors = neighbors.image_count.min()
min_wrd_neighbors = neighbors.word_count.min()
avg_img_neighbors = neighbors.image_count.mean()
avg_wrd_neighbors = neighbors.word_count.mean()

In [105]:
print "images: {} - {}, avg = {}\nwords: {} - {}, avg = {}".format(min_img_neighbors, max_img_neighbors, avg_img_neighbors,
                                                                   min_wrd_neighbors, max_wrd_neighbors, avg_wrd_neighbors)


images: 0 - 6, avg = 2.51851851852
words: 1 - 5, avg = 3.2962962963


In [108]:
neighbors.sort_values(by="word").to_csv("neighbor_counts.csv", index=False)
neighbors

,word,overlap_count,image_count,word_count
0,apple,1,1,4
1,fish,2,3,3
2,pig,1,3,4
3,duck,1,1,5
4,chair,0,3,4
5,milk,1,0,4
6,monkey,3,3,2
7,cup,1,2,4
8,frog,1,4,5
9,book,0,4,5


In [138]:
stimuli_counts = []
for root, dirs, files in os.walk("seedlings_stimuli"):
    for dir in dirs:
        num_pics = len(filter(lambda x: x.endswith(".jpg"), 
                              os.listdir(os.path.join(root, dir))))
        stimuli_counts.append((dir, num_pics))

stimuli = pd.DataFrame(stimuli_counts, columns = ["word", "num_images"])

In [139]:
stimuli = stimuli.merge(bl, left_on="word", right_on="word").sort_values(by="word")

In [141]:
stimuli.to_csv("stimuli_counts.csv", index=False)
stimuli

,word,num_images,bl_freq
0,apple,11,1110
20,baby,19,6030
15,ball,48,4503
9,bear,10,1313
25,block,21,1363
16,book,54,6299
23,bottle,13,2104
2,car,18,3066
1,cat,25,1600
13,chair,12,1767


In [153]:
stemmed_words = {}
for word in seedlings_categories:
    stemmed_words[word] = stemmer.stem(word)

childes_wordvecs = oc.wordvecs(stemmed_words.values(), "../../../../object_centrality/data_local/childes_vocab.txt", 
                                             "../../../../object_centrality/data_local/childes_vectors.npy")
childes_wordvecs.sort(key=lambda x: x[0])

In [159]:
childes_pairwise = generate_pairwise(childes_wordvecs, image_vecs)
childes_pairwise.to_csv("childes_pairwise_distances.csv", index=False)

In [160]:
childes_pairwise[:10]

,X1,word,cos_img,cos_word,animate
0,apple,baby,0.489188,0.676060,mixed
1,apple,ball,0.491353,0.597753,inanimate
2,apple,bear,0.621140,0.729320,mixed
3,apple,block,0.508780,0.755408,inanimate
4,apple,book,0.538676,0.560274,inanimate
5,apple,bottle,0.576778,0.534022,inanimate
6,apple,car,0.501118,0.654007,inanimate
7,apple,cat,0.515866,0.734291,mixed
8,apple,chair,0.542699,0.604816,inanimate
9,apple,cow,0.510026,0.794389,mixed


In [161]:
pairwise[:10]

,X1,word,cos_img,cos_word,animate
0,apple,baby,0.489188,0.680442,mixed
1,apple,ball,0.491353,0.732011,inanimate
2,apple,bear,0.621140,0.646381,mixed
3,apple,block,0.508780,0.693276,inanimate
4,apple,book,0.538676,0.633431,inanimate
5,apple,bottle,0.576778,0.686648,inanimate
6,apple,car,0.501118,0.648701,inanimate
7,apple,cat,0.515866,0.690215,mixed
8,apple,chair,0.542699,0.767749,inanimate
9,apple,cow,0.510026,0.665663,mixed
